How to stack softmax layers on top of a pretrained deep NN

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In this demo we'll be using InceptionV3 - we'll be using it as a feature extractor and stack softmax layers on top of it

In [2]:
import numpy as np
import os
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image

In [3]:
base_dir="/content/drive/MyDrive/CNN/waffle_pancakes"

While working with pretrained CNNS, we have our usual preprocessing methods to be done

We have preprocess function
and a data generator module and within the data gen class itself, im defining the preprocessing step on the images. 
Then we have the flow from directory method and supplying the path to train and test data

In [4]:
def preprocess(x):
    from keras.applications.inception_v3 import preprocess_input
    X=np.expand_dims(x,axis=0)
    X=preprocess_input(X)
    return X[0]
from keras.preprocessing import image
data_gen=image.ImageDataGenerator(rotation_range=40, ##degree range for random rotations
                                  shear_range=0.2, ##Shear Intensity (Shear angle in counter-clockwise direction in degrees)
                                  horizontal_flip=True,
                                  vertical_flip=False,
                                  zoom_range=0.2, ##Range for random zoom.
                                  fill_mode='nearest', ##Points outside the boundaries of the input are filled according to the given mode
                                  preprocessing_function=preprocess) ##function that will be applied on each input. The function will run after the image is resized and augmented
train_generator=data_gen.flow_from_directory(os.path.join(base_dir,"train"),target_size=(150,150))
valid_generator=data_gen.flow_from_directory(os.path.join(base_dir,"test"),target_size=(150,150))

Found 735 images belonging to 2 classes.
Found 389 images belonging to 2 classes.


Keras has automatically identified the number of images and classes in both train and test

Lets start assembling our model:
Inceptionv3 - feature extractor
softmax - classifier

In [5]:
## Create architecture
from keras.models import Sequential,Model
from keras.layers import Dense,Dropout

In [6]:
base_model=InceptionV3(weights='imagenet',include_top=False,input_shape=(150,150,3),pooling='avg')

87924736/87910968 [==============================] - 1s 0us/step


Inceptionv3 without top layer (removing the flattened layer) and pooling the last convolutional layer, adn pretrained weights from imagenet

In [7]:
base_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 74, 74, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 74, 74, 32)   96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 74, 74, 32)   0           batch_normalization[0][0]        
_______________________________________________________________________________________

we only have conv blocks, but no softmax layer

Now we'll use the output of our pretrained model and store it in x and stack layers on top of it

We introduce a Dense layer on the output produced by the base model and then we're putting a logreg layer. We're using Model to assemble the model



In [8]:
x = base_model.output
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 2 classes
predictions = Dense(2, activation='softmax')(x)
# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)


In [9]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 74, 74, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 74, 74, 32)   96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 74, 74, 32)   0           batch_normalization[0][0]        
______________________________________________________________________________________________

We can see two Dense layers at the bottom

Now lets ensure the weights in the convolutional blocks remain fixed while doing model training

Run a loop over each layer in the base model (inceptionv3) and make all those layers nontrainable

In [10]:
## Freeze base layer
for layer in base_model.layers:
    layer.trainable=False
model.compile(loss="categorical_crossentropy",optimizer="rmsprop",metrics=['accuracy'])

In [11]:
batch_size=32

now we call a fit_generator method and passing my train generator and valid gen that we'd defined earlier

In [12]:
model.fit_generator(
    train_generator,
    epochs=3,verbose=1,validation_data=valid_generator,shuffle=False,
    steps_per_epoch=947//batch_size,validation_steps=406//batch_size)

## 735, 389

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/3
29/29 [==============================] - 400s 14s/step - loss: 4.8971 - accuracy: 0.7265 - val_loss: 0.6608 - val_accuracy: 0.8281


We're running this for only 3 epochs because it takes time

After 3 iterations, the val accuracy is 82%. A couple more iterations and it'll reach what we'd seen in logreg 
